In [11]:
import pandas as pd
import requests
import gzip
from tqdm import tqdm
import io
import xml.etree.ElementTree as ET
import re

from tqdm import tqdm

In [2]:
# !pip install multiprocess --user
from multiprocess import Pool

In [3]:
# !pip install line_profiler
# %load_ext line_profiler

In [4]:
r = requests.get('https://apkpure.com/sitemap.xml').content

In [5]:
fp = io.StringIO(r.decode())
root = ET.parse(fp).getroot()
urls = [c[0].text for c in root]
urls = [u for u in urls if not ('default' in u or 'topics' in u or 'tag' in u or 'group' in u)]

In [6]:
def extract_apps(sitemap_url):
    try:
        r = requests.get(sitemap_url).content
    except:
        print('error', sitemap_url)
        return pd.DataFrame()
    
    fp = io.StringIO(gzip.decompress(r).decode())
    sitemap_root = ET.parse(fp).getroot()
    apps = list(sitemap_root)
    apps = [app for app in apps if 'image' in app[4].tag]
    
    df = pd.DataFrame({
        'loc': [a[0].text for a in apps],
        'lastmod': [a[1].text for a in apps],
        'changefreq': [a[2].text for a in apps],
        'priority': [a[3].text for a in apps],
        'image_loc': [a[4][0].text for a in apps],
        'name': [a[4][1].text for a in apps],
    })
    df.lastmod = pd.to_datetime(df.lastmod)
    df.priority = pd.to_numeric(df.priority)
    df['sitemap_url'] = sitemap_url
    df['category'] = re.search('(\w+)', sitemap_url.split('/')[-1]).groups()[0]
    
    return df

In [7]:
with Pool(32) as p:
    df_list = list(tqdm(p.imap_unordered(extract_apps, urls), total=len(urls)))

print('ok')
metadata = pd.concat(df_list, ignore_index=True)

100%|██████████| 7774/7774 [08:32<00:00,  9.81it/s]


ok


In [8]:
# %lprun -f extract_apps [extract_apps(url) for url in urls[:20]]

In [9]:
metadata.shape

(7744702, 8)

In [12]:
metadata.to_csv('../data/metadata.csv')